In [1]:
import pickle
import tensorflow as tf
import numpy as np
import tf_util
import gym
import load_policy

from keras.models import Sequential, load_model
from keras.layers import Dense #, Dropout, Activation, Flatten, Reshape
#from keras.utils import np_utils
from sklearn.utils import shuffle

%load_ext autoreload
%autoreload 2


/home/tiger/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
def load_task_data(filename):
    with open(filename, 'rb') as f:
        task_data = pickle.loads(f.read())
    return task_data

In [3]:
#num_rollouts = 20
expert_name = "Hopper-v1"
#expert_name = "Ant-v1"
#expert_name = "HalfCheetah-v1"
#expert_name = "Humanoid-v1"
#expert_name = "Reacher-v1"
#expert_name = "Walker2d-v1"
data_file = "data/Hopper-v1_20_data.pkl"
#data_file = "Humanoid-v1_10_data"

In [7]:
def run_clone(expert_name, expert_data_file, render = False, num_rollouts = 20):
    #expert_name: the gym expert policy name
    #render: True to render
    #num_rollouts
    
    print('loading and building expert policy')
    expert_policy_file = "./experts/" + expert_name + ".pkl"
    policy_fn = load_policy.load_policy(expert_policy_file)
    print('loaded and built')

    task_data = load_task_data(expert_data_file) #"data/" + data_file + ".pkl")
    obs_data = np.array(task_data["observations"])
    act_data = np.array(task_data["actions"])

    act_data = act_data.reshape(act_data.shape[0], act_data.shape[2])

    #create a Feedforward network useing Keras

    model = Sequential()
    model.add(Dense(96, activation = "relu", input_shape = (obs_data.shape[1],)))
    model.add(Dense(96, activation = "relu"))
    model.add(Dense(96, activation = "relu"))
    model.add(Dense(act_data.shape[1], activation = "linear"))

    model.compile(loss = "mean_squared_error", optimizer = "adam", metrics=["accuracy"])
    model.fit(obs_data, act_data, batch_size = 64, epochs = 30, verbose = 1)

    model.save('models/' + expert_name + '_cloned_model.h5')

    with tf.Session():
        tf_util.initialize()

        env = gym.make(expert_name)
        max_steps = env.spec.timestep_limit

        returns = []
        cloned_observations = []
        cloned_actions = []
        for i in range(num_rollouts):
            print('iter', i)
            obs = env.reset()

            done = False
            totalr = 0.
            steps = 0

            cloned_model = load_model('models/' + expert_name + '_cloned_model.h5')
            while not done:
                action = cloned_model.predict(obs[None, :], batch_size = 64, verbose = 0)
                cloned_observations.append(obs)
                cloned_actions.append(action)
                obs, r, done, _ = env.step(action)
                totalr += r
                steps += 1

                if render:
                    env.render()
                if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
                if steps >= max_steps:
                    break
            returns.append(totalr)

        print('returns', returns)
        print('mean return', np.mean(returns))
        print('std of return', np.std(returns))
        
        return returns


In [8]:
returns = run_clone(expert_name, data_file)

loading and building expert policy
loaded and built
Epoch 1/30
20000/20000 [==============================] - 2s 106us/step - loss: 0.1698 - acc: 0.8569
Epoch 2/30
20000/20000 [==============================] - 1s 65us/step - loss: 0.0211 - acc: 0.9353
Epoch 3/30
20000/20000 [==============================] - 1s 66us/step - loss: 0.0145 - acc: 0.9451
Epoch 4/30
20000/20000 [==============================] - 1s 63us/step - loss: 0.0106 - acc: 0.9552
Epoch 5/30
20000/20000 [==============================] - 1s 72us/step - loss: 0.0081 - acc: 0.9614
Epoch 6/30
20000/20000 [==============================] - 1s 58us/step - loss: 0.0071 - acc: 0.9630
Epoch 7/30
20000/20000 [==============================] - 2s 76us/step - loss: 0.0059 - acc: 0.9662
Epoch 8/30
20000/20000 [==============================] - 1s 63us/step - loss: 0.0052 - acc: 0.9683
Epoch 9/30
20000/20000 [==============================] - 1s 53us/step - loss: 0.0045 - acc: 0.9702
Epoch 10/30
20000/20000 [======================

[2018-04-15 18:56:26,896] From /home/tiger/Downloads/projects/CS294/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2018-04-15 18:56:26,930] From /home/tiger/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:107: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
[2018-04-15 18:56:26,990] Making new env: Hopper-v1


iter 0
100/1000
200/1000
300/1000
iter 1
100/1000
200/1000
iter 2
100/1000
200/1000
300/1000
iter 3
100/1000
200/1000
iter 4
100/1000
200/1000
iter 5
100/1000
200/1000
iter 6
100/1000
200/1000
300/1000
iter 7
100/1000
200/1000
iter 8
100/1000
200/1000
iter 9
100/1000
200/1000
300/1000
iter 10
100/1000
200/1000
300/1000
400/1000
iter 11
100/1000
200/1000
iter 12
100/1000
200/1000
300/1000
iter 13
100/1000
200/1000
iter 14
100/1000
200/1000
iter 15
100/1000
200/1000
iter 16
100/1000
200/1000
300/1000
iter 17
100/1000
200/1000
iter 18
100/1000
200/1000
300/1000
iter 19
100/1000
200/1000
300/1000
returns [1142.2000455096504, 1025.7074998333846, 1091.5494892982056, 1012.9702134223995, 1011.7602469172207, 987.84741118566069, 1087.8001557553287, 984.42417339449651, 1045.8000951330516, 1042.504440457865, 1425.8758478500986, 1039.3521516661021, 1143.0014750754353, 1015.5517126081781, 959.70416632304511, 978.40802709342745, 1164.01770002955, 986.41262674329346, 1138.5857582813976, 1162.117982297

In [9]:
print('returns', returns)
print('mean return', np.mean(returns))
print('std of return', np.std(returns))

returns [1142.2000455096504, 1025.7074998333846, 1091.5494892982056, 1012.9702134223995, 1011.7602469172207, 987.84741118566069, 1087.8001557553287, 984.42417339449651, 1045.8000951330516, 1042.504440457865, 1425.8758478500986, 1039.3521516661021, 1143.0014750754353, 1015.5517126081781, 959.70416632304511, 978.40802709342745, 1164.01770002955, 986.41262674329346, 1138.5857582813976, 1162.1179822970003]
mean return 1072.27956094
std of return 103.769228102
